# Purpose of the Notebook

In this notebook I provide you with the code for my tic-tac-toe model that I created and tried to make it perfect the game through the use of Neural nets. In this notebook you will be provided with the code of the model itself and its results against minimax and random opponents.

Model Code

In [ ]:
from typing import List, Tuple
from typing import List, Tuple
import numpy as np
from tensorflow.keras import layers, models, optimizers, losses, callbacks


def is_move_valid(board: List[List[str]], row: int, col: int) -> bool:
    return board[row][col] == ' '

def make_move(board: List[List[str]], row: int, col: int, player: str) -> None:
    board[row][col] = player

def is_winner(board: List[List[str]], player: str) -> bool:
    for i in range(3):
        if all([cell == player for cell in board[i]]) or all([board[j][i] == player for j in range(3)]):
            return True
    if (board[0][0] == board[1][1] == board[2][2] == player or board[0][2] == board[1][1] == board[2][0] == player):
        return True
    return False

def is_game_over(board: List[List[str]]) -> bool:
    return is_winner(board, player='X') or is_winner(board, player='O') or all(" " not in row for row in board)

def minimax(board: List[List[str]], depth: int, alpha: int, beta: int, maximizing_player: bool) -> int:
    if is_winner(board, "X"):
        return -1
    if is_winner(board, "O"):
        return 1
    if is_game_over(board):
        return 0

    if maximizing_player:
        best_score = float('-inf')
        for row in range(3):
            for col in range(3):
                if is_move_valid(board, row, col):
                    board[row][col] = "O"
                    score = minimax(board, depth + 1, alpha, beta, False)
                    board[row][col] = " "
                    best_score = max(score, best_score)
                    alpha = max(alpha, best_score)
                    if beta <= alpha:
                        break
        return best_score
    else:
        best_score = float('inf')
        for row in range(3):
            for col in range(3):
                if is_move_valid(board, row, col):
                    board[row][col] = "X"
                    score = minimax(board, depth + 1, alpha, beta, True)
                    board[row][col] = " "
                    best_score = min(score, best_score)
                    beta = min(beta, best_score)
                    if beta <= alpha:
                        break
        return best_score

def find_best_move(board: List[List[str]]) -> Tuple[int, int]:
    best_score = float('-inf')
    best_move = None
    alpha = float('-inf')
    beta = float('inf')
    for row in range(3):
        for col in range(3):
            if is_move_valid(board, row, col):
                board[row][col] = "O"
                score = minimax(board, 0, alpha, beta, False)
                board[row][col] = " "
                if score > best_score:
                    best_score = score
                    best_move = (row, col)
    return best_move

# Build The Neural Network
def build_model():
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(9,)),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(9, activation='softmax')
    ])
    return model


# Game Logic Functions
def is_move_valid(board: List[List[str]], row: int, col: int) -> bool:
    return board[row][col] == ' '

def make_move(board: List[List[str]], row: int, col: int, player: str) -> None:
    board[row][col] = player

def is_winner(board: List[List[str]], player: str) -> bool:
    for i in range(3):
        if all([cell == player for cell in board[i]]) or all([board[j][i] == player for j in range(3)]):
            return True
    if (board[0][0] == board[1][1] == board[2][2] == player or board[0][2] == board[1][1] == board[2][0] == player):
        return True
    return False

def is_game_over(board: List[List[str]]) -> bool:
    return is_winner(board, player='X') or is_winner(board, player='O') or all(" " not in row for row in board)

def minimax(board: List[List[str]], depth: int, alpha: int, beta: int, maximizing_player: bool) -> int:
    if is_winner(board, "X"):
        return -1
    if is_winner(board, "O"):
        return 1
    if is_game_over(board):
        return 0

    if maximizing_player:
        best_score = float('-inf')
        for row in range(3):
            for col in range(3):
                if is_move_valid(board, row, col):
                    board[row][col] = "O"
                    score = minimax(board, depth + 1, alpha, beta, False)
                    board[row][col] = " "
                    best_score = max(score, best_score)
                    alpha = max(alpha, best_score)
                    if beta <= alpha:
                        break
        return best_score
    else:
        best_score = float('inf')
        for row in range(3):
            for col in range(3):
                if is_move_valid(board, row, col):
                    board[row][col] = "X"
                    score = minimax(board, depth + 1, alpha, beta, True)
                    board[row][col] = " "
                    best_score = min(score, best_score)
                    beta = min(beta, best_score)
                    if beta <= alpha:
                        break
        return best_score

def find_best_move(board: List[List[str]]) -> Tuple[int, int]:
    best_score = float('-inf')
    best_move = None
    alpha = float('-inf')
    beta = float('inf')
    for row in range(3):
        for col in range(3):
            if is_move_valid(board, row, col):
                board[row][col] = "O"
                score = minimax(board, 0, alpha, beta, False)
                board[row][col] = " "
                if score > best_score:
                    best_score = score
                    best_move = (row, col)
    return best_move

# Generate Training Data With Adding Randomness Otherwise The Model Will Not Play As Expected
import numpy as np
from typing import List, Tuple

def generate_training_data(num_samples: int, initial_randomness: float = 0.9, final_randomness: float = 0.1, decay_rate: float = 0.995) -> List[Tuple[List[List[str]], Tuple[int, int]]]:
    data = []
    current_randomness = initial_randomness

    for i in range(num_samples):
        board = [[" " for _ in range(3)] for _ in range(3)]
        model_plays_as = np.random.choice(["X", "O"])

        while not is_game_over(board):
            empty_cells = [(row, col) for row in range(3) for col in range(3) if is_move_valid(board, row, col)]
            if not empty_cells:
                break

            if model_plays_as == "X":
                if np.random.rand() < current_randomness:
                    row, col = empty_cells[np.random.choice(len(empty_cells))]
                else:
                    row, col = find_best_move(board)
                make_move(board, row, col, "X")

                if is_game_over(board):
                    break

                best_move = find_best_move(board)
                if best_move:
                    data.append(([row[:] for row in board], best_move))
                    make_move(board, best_move[0], best_move[1], "O")
            else:
                best_move = find_best_move(board)
                if best_move:
                    make_move(board, best_move[0], best_move[1], "O")

                    if is_game_over(board):
                        break

                    if np.random.rand() < current_randomness:
                        row, col = empty_cells[np.random.choice(len(empty_cells))]
                    else:
                        row, col = find_best_move(board)
                    data.append(([row[:] for row in board], (row, col)))
        current_randomness = max(final_randomness, current_randomness * decay_rate)

        if (i + 1) % 100 == 0:
            print(f"Generated {i + 1} games... Current randomness: {current_randomness:.4f}")

    return data

# Prepare Data for TensorFlow And Train
def prepare_data(data):
    boards = []
    moves = []
    for board, move in data:
        flattened_board = [1 if cell == "X" else -1 if cell == "O" else 0 for row in board for cell in row]
        boards.append(flattened_board)
        move_index = move[0] * 3 + move[1]
        moves.append(move_index)
    return np.array(boards, dtype=np.float32), np.array(moves, dtype=np.int32)


def train_model(model, data, epochs: int = 100, batch_size: int = 32):
    boards, moves = prepare_data(data)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss=losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(boards, moves, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])

if __name__ == "__main__":
    model = build_model()
    print("Generating training data...")
    training_data = generate_training_data(2000, randomness_factor=0.9)
    print("Training the model...")
    train_model(model, training_data, epochs=50)
    model.save("tic_tac_toe_nn_Ispraven_tf.keras")
    print("Model saved!")

def prepare_data(data):
    boards = []
    moves = []
    for board, move in data:
        flattened_board = [1 if cell == "X" else -1 if cell == "O" else 0 for row in board for cell in row]
        boards.append(flattened_board)
        move_index = move[0] * 3 + move[1]
        moves.append(move_index)
    return np.array(boards, dtype=np.float32), np.array(moves, dtype=np.int32)

def train_model(model, data, epochs: int = 100, batch_size: int = 32):
    boards, moves = prepare_data(data)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss=losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(boards, moves, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])
if __name__ == "__main__":
    model = build_model()

    print("Generating training data...")
    training_data = generate_training_data(3000, randomness_factor=0.9)

    print("Training the model...")
    train_model(model, training_data, epochs=50)

    model.save("tic_tac_toe_nn_Ispraven_tf.keras")
    print("Model saved!")


# Playing first against Random opponent

In [ ]:
import numpy as np
import tensorflow as tf
import random
import time
from tensorflow.keras import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model

enhanced_model = load_model("/content/tic_tac_toe_nn_vtorobid_tf.keras", {"mse": MeanSquaredError()})
def display_board(board):
    symbols = {0: ' ', 1: 'X', -1: 'O'}
    for row in board:
        print('|'.join(symbols[cell] for cell in row))
        print('-' * 5)
def check_winner(board):
    for i in range(3):
        if abs(sum(board[i, :])) == 3:
            return np.sign(board[i, 0])
        if abs(sum(board[:, i])) == 3:
            return np.sign(board[0, i])
    if abs(sum([board[i, i] for i in range(3)])) == 3:
        return np.sign(board[0, 0])
    if abs(sum([board[i, 2 - i] for i in range(3)])) == 3:
        return np.sign(board[0, 2])
    if not np.any(board == 0):
        return 0
    return None
def get_model_move(board, model, player):
    flat_board = board.flatten().reshape(1, 9)
    predictions = model.predict(flat_board, verbose=0)
    moves = np.argsort(predictions[0])[::-1]
    for move in moves:
        row, col = divmod(move, 3)
        if board[row, col] == 0:
            return row, col
    return None
def get_random_move(board):
    available_moves = [(i, j) for i in range(3) for j in range(3) if board[i, j] == 0]
    return random.choice(available_moves)
def play_games(num_games=10):
    scores = {'wins': 0, 'draws': 0, 'losses': 0}

    for game in range(num_games):
        board = np.zeros((3, 3), dtype=int)
        current_player = 1
        print(f"\nGame {game + 1}")
        display_board(board)

        while True:
            if current_player == 1:
                row, col = get_model_move(board, enhanced_model, current_player)
            else:
                row, col = get_random_move(board)

            board[row, col] = current_player
            print(f"\nPlayer {'X' if current_player == 1 else 'O'} moved to ({row}, {col}):")
            display_board(board)

            winner = check_winner(board)
            if winner is not None:
                if winner == 1:
                    scores['wins'] += 1
                    print("Enhanced model (X) wins!")
                elif winner == -1:
                    scores['losses'] += 1
                    print("Opponent (O) wins!")
                else:
                    scores['draws'] += 1
                    print("It's a draw!")
                break

            current_player *= -1
            time.sleep(1)

    print("\nFinal Scores:")
    print(f"Wins: {scores['wins']}, Draws: {scores['draws']}, Losses: {scores['losses']}")
play_games(10)


Game 1
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
 | | 
-----
 |X| 
-----
 | | 
-----

Player O moved to (2, 0):
 | | 
-----
 |X| 
-----
O| | 
-----

Player X moved to (2, 1):
 | | 
-----
 |X| 
-----
O|X| 
-----

Player O moved to (1, 2):
 | | 
-----
 |X|O
-----
O|X| 
-----

Player X moved to (0, 0):
X| | 
-----
 |X|O
-----
O|X| 
-----

Player O moved to (0, 1):
X|O| 
-----
 |X|O
-----
O|X| 
-----

Player X moved to (2, 2):
X|O| 
-----
 |X|O
-----
O|X|X
-----
Enhanced model (X) wins!

Game 2
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
 | | 
-----
 |X| 
-----
 | | 
-----

Player O moved to (2, 2):
 | | 
-----
 |X| 
-----
 | |O
-----

Player X moved to (2, 1):
 | | 
-----
 |X| 
-----
 |X|O
-----

Player O moved to (0, 0):
O| | 
-----
 |X| 
-----
 |X|O
-----

Player X moved to (0, 1):
O|X| 
-----
 |X| 
-----
 |X|O
-----
Enhanced model (X) wins!

Game 3
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
 | | 
-----
 |X| 
-----
 | | 
-----

# Playing first against Minimax opponent

In [ ]:
import numpy as np
import tensorflow as tf
import random
import time
from tensorflow.keras import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.models import load_model

enhanced_model = load_model("/content/tic_tac_toe_nn_vtorobid_tf.keras", {"mse": MeanSquaredError()})


def display_board(board):
    symbols = {0: ' ', 1: 'X', -1: 'O'}
    for row in board:
        print('|'.join(symbols[cell] for cell in row))
        print('-' * 5)
def check_winner(board):
    for i in range(3):
        if abs(sum(board[i, :])) == 3:
            return np.sign(board[i, 0])
        if abs(sum(board[:, i])) == 3:
            return np.sign(board[0, i])
    if abs(sum([board[i, i] for i in range(3)])) == 3:
        return np.sign(board[0, 0])
    if abs(sum([board[i, 2 - i] for i in range(3)])) == 3:
        return np.sign(board[0, 2])
    if not np.any(board == 0):
        return 0
    return None
def minimax(board, depth, is_maximizing):
    winner = check_winner(board)
    if winner is not None:
        if winner == 1:
            return 1
        elif winner == -1:
            return -1
        else:
            return 0

    if is_maximizing:
        best_score = -np.inf
        for i in range(3):
            for j in range(3):
                if board[i, j] == 0:
                    board[i, j] = 1
                    score = minimax(board, depth + 1, False)
                    board[i, j] = 0
                    best_score = max(score, best_score)
        return best_score
    else:
        best_score = np.inf
        for i in range(3):
            for j in range(3):
                if board[i, j] == 0:
                    board[i, j] = -1
                    score = minimax(board, depth + 1, True)
                    board[i, j] = 0
                    best_score = min(score, best_score)
        return best_score
def get_minimax_move(board, player):
    best_score = -np.inf if player == 1 else np.inf
    best_move = None

    for i in range(3):
        for j in range(3):
            if board[i, j] == 0:
                board[i, j] = player
                score = minimax(board, 0, player == -1)
                board[i, j] = 0

                if (player == 1 and score > best_score) or (player == -1 and score < best_score):
                    best_score = score
                    best_move = (i, j)

    return best_move
def get_model_move(board, model, player):
    flat_board = board.flatten().reshape(1, 9)
    predictions = model.predict(flat_board, verbose=0)
    moves = np.argsort(predictions[0])[::-1]
    for move in moves:
        row, col = divmod(move, 3)
        if board[row, col] == 0:
            return row, col
    return None

def play_games(num_games=10):
    scores = {'wins': 0, 'draws': 0, 'losses': 0}

    for game in range(num_games):
        board = np.zeros((3, 3), dtype=int)
        current_player = 1
        print(f"\nGame {game + 1}")
        display_board(board)

        while True:
            if current_player == 1:
                row, col = get_model_move(board, enhanced_model, current_player)
            else:
                row, col = get_minimax_move(board, current_player)

            board[row, col] = current_player
            print(f"\nPlayer {'X' if current_player == 1 else 'O'} moved to ({row}, {col}):")
            display_board(board)

            winner = check_winner(board)
            if winner is not None:
                if winner == 1:
                    scores['wins'] += 1
                    print("Enhanced model (X) wins!")
                elif winner == -1:
                    scores['losses'] += 1
                    print("Minimax algorithm (O) wins!")
                else:
                    scores['draws'] += 1
                    print("It's a draw!")
                break
            current_player *= -1
            time.sleep(1)

    print("\nFinal Scores:")
    print(f"Wins: {scores['wins']}, Draws: {scores['draws']}, Losses: {scores['losses']}")
play_games(10)


Game 1
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
 | | 
-----
 |X| 
-----
 | | 
-----

Player O moved to (0, 0):
O| | 
-----
 |X| 
-----
 | | 
-----

Player X moved to (2, 1):
O| | 
-----
 |X| 
-----
 |X| 
-----

Player O moved to (0, 1):
O|O| 
-----
 |X| 
-----
 |X| 
-----

Player X moved to (0, 2):
O|O|X
-----
 |X| 
-----
 |X| 
-----

Player O moved to (2, 0):
O|O|X
-----
 |X| 
-----
O|X| 
-----

Player X moved to (1, 0):
O|O|X
-----
X|X| 
-----
O|X| 
-----

Player O moved to (1, 2):
O|O|X
-----
X|X|O
-----
O|X| 
-----

Player X moved to (2, 2):
O|O|X
-----
X|X|O
-----
O|X|X
-----
It's a draw!

Game 2
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
 | | 
-----
 |X| 
-----
 | | 
-----

Player O moved to (0, 0):
O| | 
-----
 |X| 
-----
 | | 
-----

Player X moved to (2, 1):
O| | 
-----
 |X| 
-----
 |X| 
-----

Player O moved to (0, 1):
O|O| 
-----
 |X| 
-----
 |X| 
-----

Player X moved to (0, 2):
O|O|X
-----
 |X| 
-----
 |X| 
-----

Player O moved t

# Playing second against Minimax opponent

In [ ]:
def play_games_model_second(num_games=10):
    scores = {'wins': 0, 'draws': 0, 'losses': 0}

    for game in range(num_games):
        board = np.zeros((3, 3), dtype=int)
        current_player = 1

        print(f"\nGame {game + 1}")
        display_board(board)

        while True:
            if current_player == 1:
                row, col = get_minimax_move(board, current_player)
            else:
                row, col = get_model_move(board, enhanced_model, current_player)

            board[row, col] = current_player
            print(f"\nPlayer {'X' if current_player == 1 else 'O'} moved to ({row}, {col}):")
            display_board(board)

            winner = check_winner(board)
            if winner is not None:
                if winner == 1:
                    scores['losses'] += 1
                    print("Minimax algorithm (X) wins!")
                elif winner == -1:
                    scores['wins'] += 1
                    print("Enhanced model (O) wins!")
                else:
                    scores['draws'] += 1
                    print("It's a draw!")
                break

            current_player *= -1
            time.sleep(1)

    print("\nFinal Scores:")
    print(f"Wins: {scores['wins']}, Draws: {scores['draws']}, Losses: {scores['losses']}")
play_games_model_second(10)


Game 1
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (0, 0):
X| | 
-----
 | | 
-----
 | | 
-----

Player O moved to (1, 1):
X| | 
-----
 |O| 
-----
 | | 
-----

Player X moved to (0, 1):
X|X| 
-----
 |O| 
-----
 | | 
-----

Player O moved to (0, 2):
X|X|O
-----
 |O| 
-----
 | | 
-----

Player X moved to (2, 0):
X|X|O
-----
 |O| 
-----
X| | 
-----

Player O moved to (1, 0):
X|X|O
-----
O|O| 
-----
X| | 
-----

Player X moved to (1, 2):
X|X|O
-----
O|O|X
-----
X| | 
-----

Player O moved to (2, 1):
X|X|O
-----
O|O|X
-----
X|O| 
-----

Player X moved to (2, 2):
X|X|O
-----
O|O|X
-----
X|O|X
-----
It's a draw!

Game 2
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (0, 0):
X| | 
-----
 | | 
-----
 | | 
-----

Player O moved to (1, 1):
X| | 
-----
 |O| 
-----
 | | 
-----

Player X moved to (0, 1):
X|X| 
-----
 |O| 
-----
 | | 
-----

Player O moved to (0, 2):
X|X|O
-----
 |O| 
-----
 | | 
-----

Player X moved to (2, 0):
X|X|O
-----
 |O| 
-----
X| | 
-----

Player O moved t

# Playing second against random opponent

In [ ]:
def get_random_move(board):
    empty_cells = [(i, j) for i in range(3) for j in range(3) if board[i, j] == 0]
    return random.choice(empty_cells) if empty_cells else None

def play_games_model_second_random(num_games=10):
    scores = {'wins': 0, 'draws': 0, 'losses': 0}

    for game in range(num_games):
        board = np.zeros((3, 3), dtype=int)
        current_player = 1

        print(f"\nGame {game + 1}")
        display_board(board)

        while True:
            if current_player == 1:
                row, col = get_random_move(board)
            else:
                row, col = get_model_move(board, enhanced_model, current_player)

            board[row, col] = current_player
            print(f"\nPlayer {'X' if current_player == 1 else 'O'} moved to ({row}, {col}):")
            display_board(board)

            winner = check_winner(board)
            if winner is not None:
                if winner == 1:
                    scores['losses'] += 1
                    print("Random player (X) wins!")
                elif winner == -1:
                    scores['wins'] += 1
                    print("Enhanced model (O) wins!")
                else:
                    scores['draws'] += 1
                    print("It's a draw!")
                break

            current_player *= -1
            time.sleep(1)

    print("\nFinal Scores:")
    print(f"Wins: {scores['wins']}, Draws: {scores['draws']}, Losses: {scores['losses']}")
play_games_model_second_random(10)


Game 1
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (2, 2):
 | | 
-----
 | | 
-----
 | |X
-----

Player O moved to (1, 1):
 | | 
-----
 |O| 
-----
 | |X
-----

Player X moved to (1, 0):
 | | 
-----
X|O| 
-----
 | |X
-----

Player O moved to (0, 0):
O| | 
-----
X|O| 
-----
 | |X
-----

Player X moved to (2, 0):
O| | 
-----
X|O| 
-----
X| |X
-----

Player O moved to (2, 1):
O| | 
-----
X|O| 
-----
X|O|X
-----

Player X moved to (1, 2):
O| | 
-----
X|O|X
-----
X|O|X
-----

Player O moved to (0, 1):
O|O| 
-----
X|O|X
-----
X|O|X
-----
Enhanced model (O) wins!

Game 2
 | | 
-----
 | | 
-----
 | | 
-----

Player X moved to (0, 1):
 |X| 
-----
 | | 
-----
 | | 
-----

Player O moved to (0, 0):
O|X| 
-----
 | | 
-----
 | | 
-----

Player X moved to (1, 1):
O|X| 
-----
 |X| 
-----
 | | 
-----

Player O moved to (2, 1):
O|X| 
-----
 |X| 
-----
 |O| 
-----

Player X moved to (2, 0):
O|X| 
-----
 |X| 
-----
X|O| 
-----

Player O moved to (0, 2):
O|X|O
-----
 |X| 
-----
X|O| 
-----

Play